This model will read data from a large set and Classify the Sentiment:
Positive, Negative, Natural

In [45]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
data = pd.read_csv('swiggy.csv')
data.head()


In [32]:
# Small code snippet for Tokenizer:

sentences = [
    'I am a good Boy', 
    'This is a good desk',
    'Boy is roaming around'
]

# Build vocab from the text data:
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)

# View the word index (Vocab index)
word_index = tokenizer.word_index
word_index

# Convert orignal text to sequence
text_to_sequence = tokenizer.texts_to_sequences(sentences)
text_to_sequence

# pad_sequences: This function ensures that all sequences in a list have the same length, 
# which is a requirement for feeding data into a neural network.
padded_sequence = pad_sequences(text_to_sequence, padding = 'post')
padded_sequence


array([[ 5,  6,  1,  2,  3],
       [ 7,  4,  1,  2,  8],
       [ 3,  4,  9, 10,  0]], dtype=int32)

In [14]:
data = pd.read_csv('swiggy.csv')
data.head()

,ID,Area,City,Restaurant Price,Avg Rating,Total Rating,Food Item,Food Type,Delivery Time,Review
0,1,Suburb,Ahmedabad,600,4.2,6198,Sushi,Fast Food,30-40 min,"Good, but nothing extraordinary."
1,2,Business District,Pune,200,4.7,4865,Pepperoni Pizza,Non-Vegetarian,50-60 min,"Good, but nothing extraordinary."
2,3,Suburb,Bangalore,600,4.7,2095,Waffles,Fast Food,50-60 min,Late delivery ruined it.
3,4,Business District,Mumbai,900,4.0,6639,Sushi,Vegetarian,50-60 min,Best meal I've had in a while!
4,5,Tech Park,Mumbai,200,4.7,6926,Spring Rolls,Gluten-Free,20-30 min,Mediocre experience.


In [ ]:
# Data Preprocessing:
# Cleaning and sentiment labeling:

data['Review'] = data['Review'].str.lower()
data['Review']

# good, but nothing extraordinary.
# Data still has many non-needed values we need to clear it:

data['Review'] = data['Review'].replace(r'[^a-z0-9\s]', '', regex = True)
data['Review']
# good but nothing extraordinary

# Sentiment col adding 1 for > 3.5 and 0 otherwise
data['Sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)
data['Sentiment']

# Drop the null rows:
data.dropna()

In [ ]:
# Tokenize and pad the review data and extract the target sentiment:
# Tokenize: Helps to convert text data to inter sequence
# Padding: Makes sure all the input sequences are of same length.


max_features = 5000 #Sets the maximum number of words to keep in the tokenizer
max_length = 200 # Defines the fixed length for each input sequence after padding
tokenizer = Tokenizer(num_words= max_features)
tokenizer.fit_on_texts(data['Review'])
text_to_sequence= tokenizer.texts_to_sequences(data['Review'])


# [47, 10, 11, 48],
#  [47, 10, 11, 48],
#  [71, 9, 72, 3],

X = pad_sequences(text_to_sequence, maxlen = max_length)
y = data['Sentiment'].values


X #[[ 0,  0,  0, ..., 10, 11, 48],
    #    [ 0,  0,  0, ..., 10, 11, 48],

y  #[1, 1, 1, 

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
# Splitting the data: Training, validation and test sets
# While mainting the class distribution

In [44]:
# The stratify parameter in scikit-learn's train_test_split function ensures that the resulting training and testing sets maintain 
# the same proportion of samples for each class as in the original dataset. 


# train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) : Splits data into 80% training and 20% test sets, preserving sentiment class balance
# train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train) : Further splits training data into 90% training and 10% validation sets, keeping class distribution consistent

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42, stratify = y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 42, stratify=y_train)

In [ ]:
# RNN Modeling:
# 1. Architecting Model
# 2. Compile Model
# 3. Train Model
# 4. Evaluate Model
# 5. Test Model

In [59]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(output_dim = 16, input_dim=max_features, input_length = max_length))
model.add(tf.keras.layers.SimpleRNN(units = 64, activation= 'tanh', return_sequences=False))
model.add(tf.keras.layers.Dense(units= 1, activation= 'sigmoid'))
model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs= 200, validation_data = (X_test, y_test), batch_size = 32)